## Overview ##
---
Download Basemap quads based on the mosaic name, merged and clipped to area of interest.

For example, from Explorer, I've defined my area of interest and downloaded that geoJSON.

I've selected the basemap:

global_monthly_2022_01_mosaic

and downloaded my Area of Interest.

To view the basemap, I clicked on the arrow-in-a-box icon to open that basemap in the Basemap viewer.

To see all quads in my subject area, I uploaded my Area of Interest.

326-1257,327-1257,328-1257,326-1256,327-1256,328-1256,326-1255,327-1255,328-1255


<a name="Get-a-basemap-ID"></a> 
## Get a basemap ID
---

You can get a basemap ID from Planet Explorer or the Basemap Viewer.

### Get a basemap ID from Planet Explorer
You&#8217;ll need a basemap ID to work identify what basemap to download.

To pick a basemap, you can open <a href="https://www.planet.com/explorer/#" target="_blank">Planet Explorer</a> and select a basemap from the dropdown menu in the lower left corner. (If you don't have access to basemaps, you'll see a message confirming in the lower right corner. Select "Get Access" to sign up for basemaps or speak to your customer service manager.)

Once you've selected a basemap, the Explorer updates to include that basemap in the view.

Select the search icon and time cadence, for example monthly or quarterly. Then select one of the resulting basemap IDs, such as "global_monthly_2022_05_mosaic." That's the value you'll be passing into the Orders API. 

### Get a basemap ID from the Basemap Viewer

To pick a basemap, you can open the Basemap Viewer and select a basemap from the left sidebar menu, using the filter to narrow down the basemap you want.

Once you've selected a basemap, and selected the right arrow >, the basemap ID displays, such as "global_monthly_2022_01_mosaic." That's the value you'll be passing into the Orders API.

<a name="how-to-get-started"></a> 
## Start coding

---
Next set up a session by importing needed Python packages, pulling in your API Key, and make an initial request (to retrieve the Orders API parameters) to confirm a connection with the server.

In [14]:
import planet
import os
import requests
from requests.auth import HTTPBasicAuth
import json
import time
import pathlib


<a name="setup-session"></a> 
## Setup Session Using Orders API
---
The first step is to setup a session to access the Orders API. The Orders API can be used to download full mosaics, or to download individual parts of the mosaic known as quads. 

### Create Session to Access API
---
Use your Planet API key for successful authentication. You can retrieve it from your accounts dashboard or in Explorer when selecting imagery. Save your file as an environmental variable. Here, the API key is saved to a .env file and imported into the python script.

In [15]:
API_KEY = os.environ.get('PL_API_KEY')
GCP_CREDENTIALS = os.environ.get('GCP_CREDENTIALS')
orders_url = 'https://api.planet.com/compute/ops/orders/v2' 
auth = HTTPBasicAuth(API_KEY, '')
    
# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

### Send Request and Check Status Code
---
When sending the request, we will provide one parameter. Assuming we know the name of the mosaic, we can include those details in the request to access the corresponding metadata. After sending the request, let's check the status code to ensure it was successful.

Send a get request and print out the response, which should return `<Response [200]>`.

In [16]:
response = requests.get(orders_url, auth=session.auth)
response

<Response [200]>

### Create an order packet
---
Package up the details of your order in a JSON object and make a POST request, passing in the Orders URL, your JSON, your API key, and the content-type header. 

In [17]:
order_params = {
   "name": "basemap order with geometry",
   "source_type": "basemaps",
   "order_type":"partial",
   "products": [
       {
           "mosaic_name": "global_monthly_2022_01_mosaic",
           "geometry":{
               "type": "Polygon",
               "coordinates":[
                   [
                       [ 4.607406, 52.353994 ],
                        [ 4.680005, 52.353994 ],
                        [ 4.680005, 52.395523 ],
                        [ 4.607406, 52.395523 ],
                        [ 4.607406, 52.353994 ]
                   ]
               ]
           }
      }
  ],
    "tools": [
      {"merge": {}},
      {"clip": {}}
      

  ],
 "delivery":{
      "google_cloud_storage":{
         "bucket":"notebook-practice",
         "credentials":GCP_CREDENTIALS,
         "path_prefix":"basemaps-to-cloud/"
      }


 }
}



### Print the response object
---
Retrieve the response object as text to print out if there are no errors.

In [18]:
ORDERS_API_URL = 'https://api.planet.com/compute/ops/orders/v2'
headers = {'content-type': 'application/json'}
paramRes = requests.post(ORDERS_API_URL, data=json.dumps(order_params), auth=auth, headers = {'content-type': 'application/json'})

print(paramRes.text)

{"_links":{"_self":"https://api.planet.com/compute/ops/orders/v2/501a32a9-0538-4b58-a6f0-2cca78566f5f"},"created_on":"2023-01-10T18:57:36.808Z","delivery":{"google_cloud_storage":{"bucket":"notebook-practice","credentials":"<REDACTED>","path_prefix":"basemaps-to-cloud/"}},"error_hints":[],"id":"501a32a9-0538-4b58-a6f0-2cca78566f5f","last_message":"Preparing order","last_modified":"2023-01-10T18:57:36.808Z","name":"basemap order with geometry","order_type":"partial","products":[{"geometry":{"coordinates":[[[4.607406,52.353994],[4.680005,52.353994],[4.680005,52.395523],[4.607406,52.395523],[4.607406,52.353994]]],"type":"Polygon"},"mosaic_name":"global_monthly_2022_01_mosaic"}],"source_type":"basemaps","state":"queued","tools":[{"merge":{}},{"clip":{}}]}



### Place the order
---
The following code  

In [19]:
def place_order(order_params, auth):
    response = requests.post(ORDERS_API_URL, data=json.dumps(order_params), auth=auth, headers=headers)
    print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = ORDERS_API_URL + '/' + order_id
    return order_url
order_url = place_order(order_params, session.auth)

<Response [202]>
effaf435-354a-4b4c-ab20-4df9c98df4a5


### Poll for data readiness
---
When you're ready to pull the data, you can poll for when it's ready. Here, the polling is limited to a number of loops routine.

In [22]:
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=session.auth)
        response = r.json()
        state = response['state']
        print(state)
        # print(response)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)
       
        
poll_for_success(order_url, auth)

failed
{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/effaf435-354a-4b4c-ab20-4df9c98df4a5'}, 'created_on': '2023-01-10T18:57:37.339Z', 'delivery': {'google_cloud_storage': {'bucket': 'notebook-practice', 'credentials': '<REDACTED>', 'path_prefix': 'basemaps-to-cloud/'}}, 'error_hints': [], 'id': 'effaf435-354a-4b4c-ab20-4df9c98df4a5', 'last_message': 'Status 400, {"field":null,"general":[{"message":"Unable to accept order: You must have write/delete access to bucket: notebook-practice - Upload failed - Post \\"https://storage.googleapis.com/upload/storage/v1/b/notebook-practice/o?alt=json\\u0026name=basemaps-to-cloud%2Feffaf435-354a-4b4c-ab20-4df9c98df4a5%2Fplanetverify.txt\\u0026prettyPrint=false\\u0026projection=full\\u0026uploadType=multipart\\": oauth2: cannot fetch token: 400 Bad Request\\nResponse: {\\"error\\":\\"invalid_grant\\",\\"error_description\\":\\"Invalid grant: account not found\\"}"}]}', 'last_modified': '2023-01-10T18:57:38.974Z', 'name': 'basema

In [21]:
r = requests.get(order_url, auth=session.auth)
response = r.json()
results = response['_links']['results'] 

[r['name'] for r in results]

KeyError: 'results'